## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import pandas as pd 

In [ ]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

In [ ]:
application_df.columns

In [ ]:
application_df['IS_SUCCESSFUL'].value_counts()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
dropped_df = application_df.drop(['EIN', 'NAME'], axis = 1)
dropped_df.head(30)

In [ ]:
# Find null values
for column in dropped_df.columns:
    print(f"Column {column} has {dropped_df[column].isnull().sum()} null values")

In [ ]:
# Find duplicate entries
print(f"Duplicate entries: {dropped_df.duplicated().sum()}")

In [ ]:
# application_cat = dropped_df.dtypes[dropped_df.dtypes == "object"].index.tolist()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
# dropped_df[application_cat ].nuniquie()

In [ ]:
# Determine the number of unique values in each column.
dropped_df.nunique()

In [ ]:
dropped_df.shape

In [ ]:
dropped_df.describe()

In [ ]:
dropped_df.info()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
val_counts = dropped_df['APPLICATION_TYPE'].value_counts()
val_counts

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# Transform APPLICATION_TYPE
def application_types_to_replace(application_type):
    if application_type == "T3":
        return 1
    else:
        return 0
    
# Along with replace() and map(), this is another way to encode the gender column into numbers.
dropped_df["APPLICATION_TYPE"] = dropped_df["APPLICATION_TYPE"].apply(application_types_to_replace)
dropped_df.head(20)

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(val_counts [val_counts < 1000].index)

# Replace in dataframe
for app in application_types_to_replace:
    dropped_df['APPLICATION_TYPE'] = dropped_df['APPLICATION_TYPE'].replace(app,"Other")

In [ ]:
# Check to make sure binning was successful
dropped_df["APPLICATION_TYPE"].value_counts()

In [ ]:
# try:
#     dropped_df = iter

#     for i in dropped_df:
#         print(i) 
# except TypeError as te:
#     print(te)

In [ ]:
# Scale the data
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaled_data = scaler.fit_transform(dropped_df[['APPLICATION_TYPE', 'AFFILIATION', 
#                                                'CLASSIFICATION', 'USE_CASE',
#                                                'ORGANIZATION', 'STATUS', 'INCOME_AMT', 
#                                                'ASK_AMT', 'IS_SUCCESSFUL']])

In [ ]:
# Look at CLASSIFICATION value counts for binning
clssification_value_counts = dropped_df['CLASSIFICATION'].value_counts()
clssification_value_counts 

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
clssification_value_counts[clssification_value_counts  > 1]

In [ ]:
clssification_value_counts[clssification_value_counts  > 10]

In [ ]:
clssification_value_counts[clssification_value_counts < 1000]

In [ ]:
# Determine which values to replace if counts are less than 1000
clssification_to_replace = list(clssification_value_counts [clssification_value_counts < 1000].index)

# Replace in dataframe
for cls in clssification_to_replace:
    dropped_df['CLASSIFICATION'] = dropped_df['CLASSIFICATION'].replace(cls,"Other")

In [ ]:
# Check to make sure binning was successful
dropped_df['CLASSIFICATION'].value_counts()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(dropped_df)
dummies_df.head(30)

In [ ]:
# Split our preprocessed data into our features and target arrays
X = dummies_df.drop(["IS_SUCCESSFUL"], axis='columns').values
y = dummies_df["IS_SUCCESSFUL"].values
                    

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0]) 
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation ="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn.save_weights("AlphabetSoupCharity.h5")